### Collecting Deaths from their Lists

In [1]:
import os
import pandas as pd
import re
import warnings

from collections import Counter
from pyvis.network import Network

warnings.filterwarnings('ignore')

In [2]:
kills = pd.DataFrame(columns=['universe', 'character', 'killed', 'caused', 'responsible for', 'victims'])
deaths = pd.DataFrame(columns=['universe', 'entry', 'character', 'responsible party', 'death', 'type'])
edges = pd.DataFrame(columns=['universe', 'killer', 'victim', 'direction', 'death', 'type'])

In [3]:
for path, subdirs, files in os.walk('Universes'):
	for name in files:
		if not name.endswith('.tsv'):
			continue

		file = os.path.join(path, name)
		parts = file.split('\\')

		universe = parts[1]

		entry_path_parts = parts[2:-1] + [name.replace('.tsv', '')]
		entry = ' / '.join(entry_path_parts)

		tsv = pd.read_csv(file, sep='\t')

		for i in range(len(tsv)):
			character = tsv['character'][i]
			killer = str(tsv['killed by'][i]).split(' | ')
			responsible_party = str(tsv['caused by'][i]).split(' | ')
			description = tsv['death'][i]
			type_var = tsv['type'][i]

			if killer == ['nan']:
				killer = []
			if responsible_party == ['nan']:
				responsible_party = []

			killerparty = killer + responsible_party
			if killerparty == ['nan']:
				killerparty = []

			for i in range(len(killerparty)):
				if killerparty[i] in killer:
					row = [universe, killerparty[i], 1, 0, 1, character]
					kills.loc[len(kills)] = row
				elif killerparty[i] in responsible_party:
					row = [universe, killerparty[i], 0, 1, 1, character]
					kills.loc[len(kills)] = row

				if killerparty[i] in killer:
					row = [universe, killerparty[i], character, 'direct', description, type_var]
					edges.loc[len(edges)] = row
				elif killerparty[i] in responsible_party:
					row = [universe, killerparty[i], character, 'indirect', description, type_var]
					edges.loc[len(edges)] = row

			if killerparty == []:
				row = [universe, '', character, '', description, type_var]
				edges.loc[len(edges)] = row

			row = [universe, entry, character, killerparty, description, type_var]
			deaths.loc[len(deaths)] = row

### Replacing Groups with Character Names

In [4]:
groups = pd.read_csv('Database/group.tsv', sep='\t')
remove = False

rows = []
drops = []

for i in range(len(kills)):
	universe = kills['universe'][i]
	character = kills['character'][i]
	victim = kills['victims'][i]
	pool = groups[groups['universe'] == universe]
	pool.reset_index(inplace=True)

	for j in range(len(pool)):
		if character == pool['group'][j]:
			news = str(pool['characters'][j]).split(' | ')
			for new in news:
				rows.append({
					'universe': universe,
					'character': new,
					'killed': kills.loc[i, 'killed'],
					'caused': kills.loc[i, 'caused'],
					'responsible for': kills.loc[i, 'responsible for'],
					'victims': kills.loc[i, 'victims']
				})
			drops.append(i)

		if victim == pool['group'][j]:
			news = str(pool['characters'][j]).split(' | ')
			for new in news:
				rows.append({
					'universe': universe,
					'character': kills.loc[i, 'character'],
					'killed': kills.loc[i, 'killed'],
					'caused': kills.loc[i, 'caused'],
					'responsible for': kills.loc[i, 'responsible for'],
					'victims': new
				})
			drops.append(i)

kills.drop(index=drops, inplace=True)
kills = pd.concat([kills, pd.DataFrame(rows)], ignore_index=True)


for i in range(len(deaths)):
	universe = deaths['universe'][i]
	character = deaths['character'][i]
	party = deaths['responsible party'][i]
	pool = groups[groups['universe'] == universe]
	pool.reset_index(inplace=True)

	changes = []

	for j in range(len(pool)):
		if character == pool['group'][j]:
			new = str(pool['characters'][j]).split(' | ')
			deaths.at[i, 'character'] = new

	for member in party:
		remove = False
		for j in range(len(pool)):
			if member == pool['group'][j]:
				news = str(pool['characters'][j]).split(' | ')
				changes.extend(news)
				remove = True
		if not remove:
			changes.append(member)

	deaths.at[i, 'responsible party'] = changes


rows = []
drops = []

for i in range(len(edges)):
	universe = edges['universe'][i]
	killer = edges['killer'][i]
	victim = edges['victim'][i]
	pool = groups[groups['universe'] == universe]
	pool.reset_index(inplace=True)

	for j in range(len(pool)):
		if killer == pool['group'][j]:
			news = str(pool['characters'][j]).split(' | ')
			for new in news:
				rows.append({
					'universe': universe,
					'killer': new,
					'victim': edges.loc[i, 'victim'],
					'direction': edges.loc[i, 'direction'],
					'death': edges.loc[i, 'death'],
					'type': edges.loc[i, 'type']
				})
			drops.append(i)

		if victim == pool['group'][j]:
			news = str(pool['characters'][j]).split(' | ')
			for new in news:
				rows.append({
					'universe': universe,
					'killer': edges.loc[i, 'killer'],
					'victim': new,
					'direction': edges.loc[i, 'direction'],
					'death': edges.loc[i, 'death'],
					'type': edges.loc[i, 'type']
				})
			drops.append(i)

edges.drop(index=drops, inplace=True)
edges = pd.concat([edges, pd.DataFrame(rows)], ignore_index=True)

kills.reset_index(inplace=True)
edges.reset_index(inplace=True)

### Replacing Character Names with Aliases

In [5]:
aliases = pd.read_csv('Database/alias.tsv', sep='\t')
aliases['listed'] = aliases['character'].str.split(' / ')


for i in range(len(kills)):
	universe = kills['universe'][i]
	character = kills['character'][i]
	victim = kills['victims'][i]
	pool = aliases[aliases['universe'] == universe]
	pool.reset_index(inplace=True)

	for j in range(len(pool)):
		grouped = pool['listed'][j]
		if character in grouped:
			new = pool['character'][j]
			kills.at[i, 'character'] = new
		if victim in grouped:
			new = pool['character'][j]
			kills.at[i, 'victims'] = new


for i in range(len(deaths)):
	universe = deaths['universe'][i]
	character = deaths['character'][i]
	party = deaths['responsible party'][i]
	pool = aliases[aliases['universe'] == universe]
	pool.reset_index(inplace=True)

	for j in range(len(pool)):
		grouped = pool['listed'][j]
		if character in grouped:
			new = pool['character'][j]
			deaths.at[i, 'character'] = new

		changes = dict()
		for member in party:
			if member in grouped:
				new = pool['character'][j]
				changes[member] = new
		if changes:
			party = [changes.get(member, member) for member in party]
			deaths.at[i, 'responsible party'] = party


for i in range(len(edges)):
	universe = edges['universe'][i]
	killer = edges['killer'][i]
	victim = edges['victim'][i]
	pool = aliases[aliases['universe'] == universe]
	pool.reset_index(inplace=True)

	for j in range(len(pool)):
		grouped = pool['listed'][j]
		if killer in grouped:
			new = pool['character'][j]
			edges.at[i, 'killer'] = new
		if victim in grouped:
			new = pool['character'][j]
			edges.at[i, 'victim'] = new

### Analyzing and Organizing the Deaths

In [6]:
print('----- Deadliest Universes -----')
pd.DataFrame(deaths.groupby(by=['universe']).size().sort_values(ascending=False)).reset_index().rename(columns={0:'deaths'})

----- Deadliest Universes -----


,universe,deaths
0,Elden Ring,434
1,Dark Souls,376
2,Zero Escape,206
3,Nasuverse,206
4,Project Moon,179
5,Akame ga Kill!,148
6,Hunter × Hunter,143
7,Tokyo Ghoul,140
8,Jojo's Bizarre Adventure,140
9,Attack on Titan,138


In [7]:
def join_with_counts(series):
	seen = {}
	result = []
	for v in series:
		if v not in seen:
			seen[v] = 1
			result.append(v)
		else:
			seen[v] += 1
	return ", ".join(
		f"{v} (×{seen[v]})" if seen[v] > 1 else v
		for v in result
	)

In [8]:
kills = kills.groupby(by=['universe', 'character']).aggregate({'killed':sum, 'caused':sum, 'responsible for':sum, 'victims':join_with_counts})
kills.reset_index(inplace=True)
print('----- Deadliest Killer per Universe -----')
kills.drop('victims', axis=1).sort_values(by='responsible for', ascending=False).drop_duplicates('universe').reset_index(drop=True)

----- Deadliest Killer per Universe -----


,universe,character,killed,caused,responsible for
0,Elden Ring,The Tarnished,344,1,345
1,Zero Escape,Delta / Brother / Q / Zero II,45,96,141
2,Dark Souls,Bearer of the Curse,125,0,125
3,Project Moon,Roland / The Black Silence,106,0,106
4,Hollow Knight,The Knight / Lord of Shades,73,0,73
5,Lords of the Fallen,The Lampbearer / The Deathless One,43,2,45
6,Assassin's Creed,Ezio Auditore da Firenze,34,6,40
7,Akame ga Kill!,Akame,40,0,40
8,Star Wars,Cal Kestis,38,0,38
9,Danganronpa,Monokuma,20,18,38


In [9]:
deaths['character'] = deaths['character'].apply(lambda x: x if isinstance(x, list) else [x])
deaths_exploded = deaths.explode('character', ignore_index=True)
deaths_count = deaths_exploded.groupby(['universe', 'character']).size().reset_index(name='deaths')

kills_count = kills.copy()
kills_count.drop(['victims'], axis=1, inplace=True)
kills_count.columns = ['universe', 'character', 'killed', 'caused', 'responsible for']

summary = pd.merge(deaths_count, kills_count, left_on=['universe', 'character'], right_on=['universe', 'character'], how='outer')
summary.fillna(0, inplace=True)
summary.sort_values(by=['universe', 'character'], inplace=True)

summary = summary.round(decimals=2).astype(object)
summary = summary.astype(str)
summary = summary.replace(to_replace = "\.0+$",value = "", regex = True)

In [10]:
deaths['resulting status'] = ''

for i in range(len(deaths)):
	type = str(deaths['type'][i]).split(' | ')

	conditions = ["variable"]
	if re.search('|'.join(conditions), str(type)):
		dependent = True
	else:
		dependent = False

	conditions = ["unavoidable"]
	if re.search('|'.join(conditions), str(type)):
		dependent = False

	conditions = ["temporary", "non-canon"]
	if re.search('|'.join(conditions), str(type)):
		alive = True
	else:
		alive = False

	conditions = ["physical", "undead"]
	if re.search('|'.join(conditions), str(type)):
		extant = True
	else:
		extant = False

	if alive:
		deaths['resulting status'][i] = 'alive'
	elif extant and not alive:
		deaths['resulting status'][i] = 'extant'
	elif dependent and not extant and not alive:
		deaths['resulting status'][i] = 'dependent'
	if not dependent and not extant and not alive:
		deaths['resulting status'][i] = 'deceased'

In [11]:
priority = {"deceased": 1, "extant": 2, "dependent": 3, "alive": 4}

status = deaths.copy()
status["character"] = status["character"].apply(lambda x: x if isinstance(x, list) else [x])
status = status.explode("character", ignore_index=True)

status["priority"] = status["resulting status"].map(priority)

status = status.loc[status.groupby(["universe", "character"])["priority"].idxmin()]
status = status.drop(columns=["priority", "responsible party", "death", "type"]).reset_index(drop=True)
status.rename(columns={"resulting status": "last known status"}, inplace=True)

summary = pd.merge(summary, status, on=["universe", "character"], how="outer")
summary["last known status"] = summary["last known status"].fillna("alive")

In [12]:
deaths.to_csv('deaths.tsv', sep='\t', index=False)
kills.to_csv('kills.tsv', sep='\t', index=False)
summary.to_csv('summary.tsv', sep='\t', index=False)

### Generating a Summary of Universes

In [13]:
deaths_summary = deaths.drop(["character", "responsible party", "death", "type", "resulting status"], axis=1)
deaths_summary['deaths'] = 1
deaths_summary = deaths_summary.groupby(["universe", "entry"]).aggregate({"deaths": sum})
deaths_summary = deaths_summary.reset_index()

final_rows = []

grand_total = deaths_summary["deaths"].sum()
final_rows.append({
	"universe": "**\***",
	"entry": "**\***",
	"deaths": f"**{grand_total}**"
})

for universe, group in deaths_summary.groupby("universe"):
	universe_total = group["deaths"].sum()

	final_rows.append({
		"universe": f"**{universe}**",
		"entry": "**\***",
		"deaths": f"**{universe_total}**"
	})

	final_rows.extend(group.to_dict(orient='records'))

final_table = pd.DataFrame(final_rows)
deaths_table = final_table.to_markdown(tablefmt='github', index=False)

with open('README.md', 'r+') as f:
	f.seek(0)
	f.write(deaths_table)
	f.truncate()

### Visualizing Death Links using PyVis

In [14]:
status = summary.copy()
status.drop(columns=["deaths", "killed", "caused", "responsible for"], inplace=True)
status.rename(columns={"last known status": "color"}, inplace=True)

colors = {"alive" : "#107a06",
		  "extant" : "#0990b5",
		  "dependent" : "#631da3",
		  "deceased" : "#730a1c"}

status["color"].replace(colors, inplace=True)
status["character"] += " [" + status["universe"] + "]"

In [15]:
edges['killer'] += " [" + edges['universe'] + "]"
edges.loc[edges['killer'].str.startswith(' ['), 'killer'] = ''
edges['victim'] += " [" + edges['universe'] + "]"

nodes = pd.DataFrame()
nodes['character'] = pd.concat([edges['victim'], edges['killer']]).drop_duplicates().sort_values().reset_index(drop=True)

nodes = pd.merge(nodes, status, left_on='character', right_on='character', how='outer')
nodes.sort_values(by=['character', 'color'], inplace=True)

deaths["character"] = deaths["character"].apply(lambda x: x if isinstance(x, list) else [x])
deaths = deaths.explode("character", ignore_index=True)

descriptions = deaths.groupby(by=['universe', 'character']).aggregate({'death':'\n'.join})
descriptions.reset_index(inplace=True)
descriptions['character'] += " [" + descriptions['universe'] + "]"
descriptions.drop(columns=['universe'], inplace=True)

nodes = pd.merge(nodes, descriptions, left_on=['character'], right_on=['character'], how='outer')

profiles = pd.DataFrame(columns=['universe', 'character', 'profile'])

In [16]:
def make_network(universe):
	net = Network(height="100vh", width="100%", bgcolor="#111111", directed=True, notebook=False)
	net.force_atlas_2based(overlap=1, damping=0.5)

	current_edges = edges[edges['universe'] == universe]
	current_nodes = nodes[nodes['character'].str.endswith('[' + universe + ']')]

	sources, targets, direction = current_edges['killer'], current_edges['victim'], current_edges['direction']
	description = current_edges['death']
	edge_data = zip(sources, targets, direction, description)

	for i in range(len(current_nodes)):
		victim = current_nodes.iloc[i]['character']
		color = current_nodes.iloc[i]['color']
		label = victim.split(" [")[0]
		description = current_nodes.iloc[i]['death']
		title = label + "\n\nCharacter Profile :" + "\n\n\n"

		if description == description:
			value = description.count('\n') + 1
			if value != 1:
				plural = 's'
			else:
				plural = ''

			title += str(value) + " Death" + plural + " :\n\n" + description + "\n\n"
		else:
			title += ''

		if victim != '':
			font = f"15px baskerville {color} sans-serif"
			net.add_node(victim, label=label, title=title, color=color, font=font)

	for src, dst, dir, des in edge_data:
		if src != '':
			if dir == 'direct':
				net.add_edge(src, dst, title=des, color='#e8ac87', width=1, dashes=False)
			elif dir == 'indirect':
				net.add_edge(src, dst, title=des, color='#cccccc', width=1, dashes=True)

	neighbor_map = net.get_adj_list()
	edges_map = net.get_edges()

	pair_counts = Counter((e['from'], e['to']) for e in edges_map)

	def format_neighbors_with_counts(node_id):
		raw = neighbor_map.get(node_id, [])
		seen = []
		counts = {}
		for nbr in raw:
			if nbr not in counts:
				counts[nbr] = pair_counts.get((node_id, nbr), 0)
				seen.append(nbr)
		formatted = []
		for nbr in seen:
			cnt = counts[nbr]
			if cnt > 1:
				formatted.append(f"{nbr} (×{cnt})")
			else:
				formatted.append(nbr)
		return "\n".join(sorted(formatted))

	for node in net.nodes:
		nid = node['id']
		count_unique = len(neighbor_map.get(nid, []))
		if count_unique:
			plural = 's' if count_unique != 1 else ''
			node['title'] += f"{count_unique} Victim{plural} :\n\n"
			node['title'] += format_neighbors_with_counts(nid)
			node['title'] = re.sub(r' \[.*?\]', '', node['title'])
		node['value'] = count_unique

	net.inherit_edge_colors(False)
	net.set_edge_smooth('dynamic')

	net.save_graph('Networks/' + universe + '.html')

	for node in net.nodes:
		universe = node['id'].split(' [')[1].split(']')[0]
		character = node['id'].split(' [')[0]
		profile = node['title']
		row = [universe, character, profile]
		profiles.loc[len(profiles)] = row

In [17]:
for universe in kills['universe'].unique():
	make_network(universe)